In [1]:
from dataclasses import dataclass
from typing import List

In [2]:
@dataclass
class Component_Type():
    name: str
    requirements: str

@dataclass
class Component():
    content: str
    version: int
    type: Component_Type

    def revision(self, new_content: str):
        self.content = new_content
        self.version += 1

@dataclass
class Concept():
    name: str
    version: int
    description: str
    components: List[Component]

    def add_component(self, component: Component):
        self.version += 1
        self.components.append(component)
    
    def revise_component(self, component: Component, new_content: str):
        component.revision(new_content)
        self.version += 1

@dataclass
class Template():
    name: str
    description: str
    pattern: List[Component_Type]

    def check_concept(self, concept: Concept):
        missing_components = []
        for component_type in self.pattern:
            found = False
            for component in concept.components:
                if component.type == component_type:
                    found = True
                    break
            if not found:
                missing_components.append(component_type.name)
        print("Missing Components:", missing_components)
        return missing_components == []


@dataclass
class Document():
    name: str
    version: int
    components: List[Component]
    concept: Concept
    template: Template

    def export(self):
        return "\n".join([c.content for c in self.components])


def document_factory(concept:Concept, template: Template) -> Document:
    #check if concept is compatible with template
    if not template.check_concept(concept):
        raise ValueError("Concept is not compatible with template")
    
    components = []
    for component_type in template.pattern:
        for component in concept.components:
            if component.type == component_type:
                components.append(component)
                break

    return Document(name=concept.name + template.name, version=concept.version, components=components, concept=concept, template=template)


    

In [3]:
#define component types
legalese = Component_Type("Legalese", "Legal requirements")
summary = Component_Type("Summary", "Summary of the concept")
quadchart = Component_Type("Quadchart", "Quadchart of the concept")
approach = Component_Type("Approach", "Approach to the concept")
feasibilityRisksAssumptions = Component_Type("Feasibility, Risks, Assumptions", "description of the feasibility, risks and assumptions of the concept")
successCriteriaMilestones = Component_Type("Success Criteria and Milestones", "Success criteria and milestones of the concept")

#define a template
TwoPager = Template("Two-Pager", "A 2 page document", [legalese, summary, quadchart])
SixPager = Template("Six-Pager", "A 6 page document", [legalese, summary, quadchart, approach, feasibilityRisksAssumptions, successCriteriaMilestones])

#define sample components
legalese_component = Component("Legalese content Lipsum etc", 1, legalese)
summary_component = Component("Summary content Lipsum etc", 1, summary)
quadchart_component = Component("Quadchart content Lipsum etc", 1, quadchart)
approach_component = Component("Approach content Lipsum etc", 1, approach)
feasibilityRisksAssumptions_component = Component("Feasibility, Risks, Assumptions content Lipsum etc", 1, feasibilityRisksAssumptions)
successCriteriaMilestones_component = Component("Success Criteria and Milestones content Lipsum etc", 1, successCriteriaMilestones)

#define a concept
my_concept = Concept("Concept 1", 1, "Description of concept 1", [legalese_component, summary_component, quadchart_component, approach_component])


In [4]:
TwoPager.check_concept(my_concept)

Missing Components: []


True

In [5]:
#define a document
twopagedocument = document_factory(my_concept, TwoPager)

Missing Components: []


In [6]:
print(twopagedocument.export())

Legalese content Lipsum etc
Summary content Lipsum etc
Quadchart content Lipsum etc


In [7]:
#recise a component
my_concept.revise_component(summary_component, "New summary Lipsum etc")
new_twopagedocument = document_factory(my_concept, TwoPager)


Missing Components: []


In [8]:

print(new_twopagedocument.export())

Legalese content Lipsum etc
New summary Lipsum etc
Quadchart content Lipsum etc


In [9]:
SixPager.check_concept(my_concept) #should raise an error

Missing Components: ['Feasibility, Risks, Assumptions', 'Success Criteria and Milestones']


False

In [10]:
feasibilityRisksAssumptions_component = Component("Feasibility, Risks, Assumptions content Lipsum etc", 1, feasibilityRisksAssumptions)
my_concept.add_component(feasibilityRisksAssumptions_component)

successCriteriaMilestones_component = Component("Sucess Criteria and Milestones content Lipsum etc", 1, successCriteriaMilestones)
my_concept.add_component(successCriteriaMilestones_component)


In [11]:
my_concept

Concept(name='Concept 1', version=4, description='Description of concept 1', components=[Component(content='Legalese content Lipsum etc', version=1, type=Component_Type(name='Legalese', requirements='Legal requirements')), Component(content='New summary Lipsum etc', version=2, type=Component_Type(name='Summary', requirements='Summary of the concept')), Component(content='Quadchart content Lipsum etc', version=1, type=Component_Type(name='Quadchart', requirements='Quadchart of the concept')), Component(content='Approach content Lipsum etc', version=1, type=Component_Type(name='Approach', requirements='Approach to the concept')), Component(content='Feasibility, Risks, Assumptions content Lipsum etc', version=1, type=Component_Type(name='Feasibility, Risks, Assumptions', requirements='description of the feasibility, risks and assumptions of the concept')), Component(content='Sucess Criteria and Milestones content Lipsum etc', version=1, type=Component_Type(name='Success Criteria and Miles

In [12]:

SixPager.check_concept(my_concept)

Missing Components: []


True

In [13]:

sixpage_document = document_factory(my_concept, SixPager)

print(sixpage_document.export())

Missing Components: []
Legalese content Lipsum etc
New summary Lipsum etc
Quadchart content Lipsum etc
Approach content Lipsum etc
Feasibility, Risks, Assumptions content Lipsum etc
Sucess Criteria and Milestones content Lipsum etc


In [14]:
#revise a component
my_concept.revise_component(summary_component, "New NEW summary Lipsum etc")
newnew_twopagedocument = document_factory(my_concept, TwoPager)

Missing Components: []


In [15]:
print(newnew_twopagedocument.export())

Legalese content Lipsum etc
New NEW summary Lipsum etc
Quadchart content Lipsum etc


In [16]:
#create a new concept with new components but the same component types
new_legalese_component = Component("New Legalese content Lipsum etc", 1, legalese)
new_summary_component = Component("New Summary content Lipsum etc", 1, summary)
new_quadchart_component = Component("New Quadchart content Lipsum etc", 1, quadchart)
new_approach_component = Component("New Approach content Lipsum etc", 1, approach)
new_feasibilityRisksAssumptions_component = Component("New Feasibility, Risks, Assumptions content Lipsum etc", 1, feasibilityRisksAssumptions)
new_successCriteriaMilestones_component = Component("New Success Criteria and Milestones content Lipsum etc", 1, successCriteriaMilestones)

new_concept = Concept("Concept 2", 1, "Description of concept 2", [new_legalese_component, new_summary_component, new_quadchart_component, new_approach_component, new_feasibilityRisksAssumptions_component, new_successCriteriaMilestones_component])

new_sixpage_document = document_factory(new_concept, SixPager)

Missing Components: []


In [18]:
print(new_sixpage_document.export())

New Legalese content Lipsum etc
New Summary content Lipsum etc
New Quadchart content Lipsum etc
New Approach content Lipsum etc
New Feasibility, Risks, Assumptions content Lipsum etc
New Success Criteria and Milestones content Lipsum etc
